# Build Python Models to Predict Machine Failure
- XGBoost:  Serialize with Pickle and ONNX
- Tensorflow: Serialize with H5

In [1]:
import csv
import pandas as pd
import numpy as np
import os, shutil # used to create necessary folders
import pickle
import json
import sklearn
import tensorflow as tf
from xgboost import XGBClassifier 
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
 
 


2022-06-16 19:31:17.943637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 19:31:17.943782: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
machine = pd.read_csv('Data_orig/machine_failure.csv')

machine.head()

,MACHINE_ID,FAILURE,DAILY_PRODUCTION,MONTHLY_PRODUCTION,PROD_TARGET,REASON,MONTHS_IN_OPERATION,NEGATIVE_RATING,DAYS_OUT_OF_SERVICE,MACHINE_AGE,SENSOR_INQUIRIES,NUM_SENSORS,SENSOR_RATIO,INDUSTRY
0,772418,1,1100,25860.0,39025.0,TruckPr,10.5,0.0,0.0,94.366667,1.0,9.0,NaN,Other
1,477724,1,1300,70053.0,68400.0,TruckPr,7.0,0.0,2.0,121.833333,0.0,14.0,NaN,Other
2,150746,1,1500,13500.0,16700.0,TruckPr,4.0,0.0,0.0,149.466667,1.0,10.0,NaN,Other
3,108584,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,321534,0,1700,97800.0,112000.0,TruckPr,3.0,0.0,0.0,93.333333,0.0,14.0,NaN,FLEET


In [3]:
machine.dtypes
 

MACHINE_ID               int64
FAILURE                  int64
DAILY_PRODUCTION         int64
MONTHLY_PRODUCTION     float64
PROD_TARGET            float64
REASON                  object
MONTHS_IN_OPERATION    float64
NEGATIVE_RATING        float64
DAYS_OUT_OF_SERVICE    float64
MACHINE_AGE            float64
SENSOR_INQUIRIES       float64
NUM_SENSORS            float64
SENSOR_RATIO           float64
INDUSTRY                object
dtype: object

In [4]:
target         = ["FAILURE"]
numeric_inputs = ['DAILY_PRODUCTION', 'MONTHLY_PRODUCTION', 'PROD_TARGET', 'MONTHS_IN_OPERATION', 
                  'NEGATIVE_RATING', 'DAYS_OUT_OF_SERVICE', 'MACHINE_AGE', 'SENSOR_INQUIRIES','NUM_SENSORS','SENSOR_RATIO']
class_inputs   = ['REASON', 'INDUSTRY']
 
impute_values = machine[numeric_inputs].mean()
pickle.dump(impute_values, open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle','wb'))
   
    
machine           =machine.fillna(impute_values)
machine.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
machine.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)

 
x              = machine.drop(['FAILURE', 'MACHINE_ID'], axis=1)
y              = machine[target].astype("category")

 

In [5]:
machine2 =machine.head()
machine2.drop(['FAILURE'], axis=1, inplace=True)
machine2.to_csv('Data_orig/machine2_ds2_test.csv', index=False)
 
    

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Split Dataset Into Training and Test Set

In [6]:
# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

print("train size: " + str(x_train.shape))
print("test size: " + str(x_test.shape))
print("train sample")

 
 
 
 

train size: (4172, 12)
test size: (1788, 12)
train sample


# Custom preprocessing for categorical inputs to use in XGB model

In [7]:
 
ohe = OneHotEncoder(sparse = False, handle_unknown="ignore")
enc = ohe.fit_transform(x_train[class_inputs])
enc2 = ohe.transform(x_test[class_inputs])

 
x_train[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc, index=x_train.index)
x_train.drop(class_inputs,1,inplace=True)

x_test[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc2, index=x_test.index)
x_test.drop(class_inputs,1,inplace=True)


print(x_train.shape)
print(x_test.shape)


 
 


(4172, 18)
(1788, 18)


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3776711327.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_train.drop(class_inputs,1,inplace=True)
/tmp/ipykernel_9143/3776711327.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_test.drop(class_inputs,1,inplace=True)


In [8]:
#Pickle the one hot encoding
pickle.dump(ohe, open('/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle', 'wb'))

# Standardize Inputs with Standard Scaling

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#scaler = MinMaxScaler()
# fit scaler on the training dataset
#scaler.fit(X_train)
#X_train_scaled = scaler.transform(X_train)
pickle.dump(scaler, open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'wb'))

type(x_train)

numpy.ndarray

# Train the XGBoost Model

In [10]:
# Create XGBoost classifer object

 
# x_train_array = x_train.values
# y_train_array = y_train.values
    
booster = xgb.XGBClassifier(max_depth=3,
                        booster='dart',
                        eta=0.3,
                        silent=1,
                        n_estimators=100)


# Train XGBoost Classifer
booster.fit(x_train,y_train)

modelName = "MACHINE_XGBoost"
 

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

[19:31:27] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:31:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Train the Tensorflow Model

In [11]:
# Train Tensorflow Classified with Kera API 
clf_tens = tf.keras.models.Sequential([tf.keras.layers.Dense(units=32, activation = 'relu',input_shape=(x_train.shape[-1],)),
                                 tf.keras.layers.Dropout(0.9),
                                 tf.keras.layers.Dense(units=16, activation = 'relu'),
                                 tf.keras.layers.Dropout(0.9),
                                 tf.keras.layers.Dense(units=8, activation = 'relu'),
                                 tf.keras.layers.Dropout(0.9),
                               
                                 # use sigmoid activation for binary target, softmax did not work
                                 tf.keras.layers.Dense(1, activation = 'sigmoid')])

# use binary_crossentropy loss function for binary target
clf_tens.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0006),
              loss='binary_crossentropy',
              metrics = ['acc'])

# train a model
clf_tens.fit(x_train,y_train, epochs=500,batch_size=int(len(x_train)/10),verbose=1,validation_data=(x_test, y_test))

modelName2 = "MACHINE_Tensorflow"

2022-06-16 19:31:28.148187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-16 19:31:28.148237: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-16 19:31:28.148273: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eeclxvm068.exnet.sas.com): /proc/driver/nvidia/version does not exist
2022-06-16 19:31:28.148774: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/keras/optimizer_v2/rmsp

Epoch 1/500
11/11 [==============================] - 1s 22ms/step - loss: 3.1915 - acc: 0.7601 - val_loss: 0.8546 - val_acc: 0.4849
Epoch 2/500
11/11 [==============================] - 0s 8ms/step - loss: 3.2990 - acc: 0.7589 - val_loss: 0.8229 - val_acc: 0.5185
Epoch 3/500
11/11 [==============================] - 0s 5ms/step - loss: 2.5513 - acc: 0.7656 - val_loss: 0.8031 - val_acc: 0.5447
Epoch 4/500
11/11 [==============================] - 0s 5ms/step - loss: 2.7519 - acc: 0.7637 - val_loss: 0.7842 - val_acc: 0.5576
Epoch 5/500
11/11 [==============================] - 0s 5ms/step - loss: 2.5264 - acc: 0.7658 - val_loss: 0.7666 - val_acc: 0.5839
Epoch 6/500
11/11 [==============================] - 0s 5ms/step - loss: 2.7320 - acc: 0.7617 - val_loss: 0.7489 - val_acc: 0.6012
Epoch 7/500
11/11 [==============================] - 0s 5ms/step - loss: 2.6924 - acc: 0.7622 - val_loss: 0.7348 - val_acc: 0.6158
Epoch 8/500
11/11 [==============================] - 0s 5ms/step - loss: 2.5092 - 

11/11 [==============================] - 0s 4ms/step - loss: 0.7131 - acc: 0.7999 - val_loss: 0.5818 - val_acc: 0.7919
Epoch 64/500
11/11 [==============================] - 0s 5ms/step - loss: 0.6764 - acc: 0.7975 - val_loss: 0.5810 - val_acc: 0.7919
Epoch 65/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6878 - acc: 0.7996 - val_loss: 0.5798 - val_acc: 0.7919
Epoch 66/500
11/11 [==============================] - 0s 4ms/step - loss: 0.7116 - acc: 0.7958 - val_loss: 0.5788 - val_acc: 0.7919
Epoch 67/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6678 - acc: 0.7982 - val_loss: 0.5777 - val_acc: 0.7919
Epoch 68/500
11/11 [==============================] - 0s 5ms/step - loss: 0.6974 - acc: 0.8011 - val_loss: 0.5770 - val_acc: 0.7919
Epoch 69/500
11/11 [==============================] - 0s 5ms/step - loss: 0.6598 - acc: 0.7982 - val_loss: 0.5763 - val_acc: 0.7919
Epoch 70/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6699 - acc: 0

Epoch 125/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5723 - acc: 0.8027 - val_loss: 0.5362 - val_acc: 0.7919
Epoch 126/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5395 - acc: 0.8023 - val_loss: 0.5357 - val_acc: 0.7919
Epoch 127/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5448 - acc: 0.8030 - val_loss: 0.5353 - val_acc: 0.7919
Epoch 128/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5418 - acc: 0.8042 - val_loss: 0.5347 - val_acc: 0.7919
Epoch 129/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5561 - acc: 0.8023 - val_loss: 0.5346 - val_acc: 0.7919
Epoch 130/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5366 - acc: 0.8035 - val_loss: 0.5342 - val_acc: 0.7919
Epoch 131/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5391 - acc: 0.8027 - val_loss: 0.5336 - val_acc: 0.7919
Epoch 132/500
11/11 [==============================] - 0s 5ms/step - 

11/11 [==============================] - 0s 5ms/step - loss: 0.5114 - acc: 0.8035 - val_loss: 0.5165 - val_acc: 0.7919
Epoch 187/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5133 - acc: 0.8049 - val_loss: 0.5163 - val_acc: 0.7919
Epoch 188/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5057 - acc: 0.8049 - val_loss: 0.5162 - val_acc: 0.7919
Epoch 189/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5044 - acc: 0.8042 - val_loss: 0.5161 - val_acc: 0.7919
Epoch 190/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5063 - acc: 0.8039 - val_loss: 0.5158 - val_acc: 0.7919
Epoch 191/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5044 - acc: 0.8047 - val_loss: 0.5158 - val_acc: 0.7919
Epoch 192/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5055 - acc: 0.8049 - val_loss: 0.5156 - val_acc: 0.7919
Epoch 193/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5032 -

Epoch 248/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4994 - acc: 0.8039 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 249/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4982 - acc: 0.8039 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 250/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4966 - acc: 0.8044 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 251/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4970 - acc: 0.8044 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 252/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4976 - acc: 0.8047 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 253/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4973 - acc: 0.8042 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 254/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4980 - acc: 0.8042 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 255/500
11/11 [==============================] - 0s 4ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.4965 - acc: 0.8047 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 310/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4943 - acc: 0.8044 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 311/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4954 - acc: 0.8051 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 312/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4950 - acc: 0.8047 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 313/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4964 - acc: 0.8044 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 314/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4942 - acc: 0.8054 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 315/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4950 - acc: 0.8039 - val_loss: 0.5113 - val_acc: 0.7919
Epoch 316/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4943 -

Epoch 371/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4937 - acc: 0.8047 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 372/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4928 - acc: 0.8047 - val_loss: 0.5117 - val_acc: 0.7919
Epoch 373/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4929 - acc: 0.8044 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 374/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4925 - acc: 0.8049 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 375/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4959 - acc: 0.8032 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 376/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4935 - acc: 0.8047 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 377/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4935 - acc: 0.8056 - val_loss: 0.5116 - val_acc: 0.7919
Epoch 378/500
11/11 [==============================] - 0s 5ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.4947 - acc: 0.8039 - val_loss: 0.5119 - val_acc: 0.7919
Epoch 433/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4934 - acc: 0.8037 - val_loss: 0.5119 - val_acc: 0.7919
Epoch 434/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4928 - acc: 0.8054 - val_loss: 0.5119 - val_acc: 0.7919
Epoch 435/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4936 - acc: 0.8042 - val_loss: 0.5119 - val_acc: 0.7919
Epoch 436/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4927 - acc: 0.8054 - val_loss: 0.5119 - val_acc: 0.7919
Epoch 437/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4925 - acc: 0.8051 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 438/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4951 - acc: 0.8039 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 439/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4922 -

Epoch 494/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4970 - acc: 0.8035 - val_loss: 0.5120 - val_acc: 0.7919
Epoch 495/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4935 - acc: 0.8051 - val_loss: 0.5120 - val_acc: 0.7919
Epoch 496/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4906 - acc: 0.8063 - val_loss: 0.5120 - val_acc: 0.7919
Epoch 497/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4913 - acc: 0.8066 - val_loss: 0.5120 - val_acc: 0.7919
Epoch 498/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4946 - acc: 0.8047 - val_loss: 0.5120 - val_acc: 0.7919
Epoch 499/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4927 - acc: 0.8044 - val_loss: 0.5120 - val_acc: 0.7919
Epoch 500/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4929 - acc: 0.8047 - val_loss: 0.5120 - val_acc: 0.7919


# Pickle the trained XGB Model and H5 for Tensorflow Model

In [12]:
#pickle.dump(model, open(project_dir+"/HMEQ_XGB.pickle", 'wb'))
pickle.dump(booster, open("/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName + ".pickle", 'wb'))

clf_tens.save(filepath = "/sasinside/userdata/gegrab/resources/hmeq" + '/' + modelName2 + '.h5')

# Write the XGBoost Model to ONNX


In [13]:
model     = booster
modelName3 = "MACHINE_ONNX_XGBoost"

column_count = x_train.shape[1] #second position
column_count



18

In [14]:
import onnx
from onnxmltools.convert.common import data_types

initial_type = [('float_input', data_types.FloatTensorType([1, column_count]))]

In [15]:
from onnxmltools.convert import convert_xgboost

booster_onnx = convert_xgboost(booster, initial_types=initial_type)
onnx.save(booster_onnx, "/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName3 +".onnx")
 

In [16]:
# import os
# os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'


# import swat
# from sasctl import Session
# from sasctl.tasks import register_model, publish_model

# Connect to the CAS server
 
# s=swat.CAS("eeclxvm068", 5570, 'gegrab','Demopw123')


# onnxfile = "/sasinside/userdata/gegrab/resources/hmeq/MACHINE_ONNX_XGBoost.onnx"
# with open(onnxfile,"rb") as file:
#     blob = file.read()
# blob_ = swat.blob(blob)

In [17]:
# swat.aStore.extract(rstore="onnxstore")

# Write Out Parameters for Copying

In [18]:
# Define input variables and perform formatting to match with score code file

input_params = ''
for col in machine.columns:
    input_params += col
    if col != machine.columns[-1]:
        input_params += ', '

input_cols = ''
for col in machine.columns:
    input_cols += "\"" + col + "\""
    if col != machine.columns[-1]:
        input_cols += ', '


print(input_cols)
print("")
print(input_params)

"MACHINE_ID", "FAILURE", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"

MACHINE_ID, FAILURE, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY


# Test ONNX Scoring Code

In [19]:
def scoreONNX(MACHINE_ID,DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY ):
    "Output: P_FAILURE"
        
    
    
    import onnxruntime as rt
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY]],
                              columns = ["MACHINE_ID", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
    inputArray.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)
    
    

    inputArray.drop(['MACHINE_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = rt.InferenceSession("/sasinside/userdata/gegrab/resources/hmeq/MACHINE_ONNX_XGBoost.onnx")
    
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    probabilities = sess.get_outputs()[1].name
    
    for x in test_data:
        pred_onnx = sess.run([probabilities], {input_name: np.expand_dims(x, axis=0).astype(np.float32)})[0]
    

    P_FAILURE = float(pred_onnx[0][0])
    
    
     
    return (P_FAILURE)


 

In [20]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/machine2_ds2_test.csv') 
X_test2 
 
# gene2 = gene.drop(['BAD','Loan_ID'], axis=1)
 
for i in range(5):
    print(scoreONNX(**X_test2.iloc[i]))

0.050464928150177
0.09749710559844971
0.03160446882247925
0.1253322958946228
0.23529398441314697


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3088785506.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3088785506.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

# Write Out ONNX Score Code

In [21]:
score_code = """
def scoreONNX(MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY ):
    "Output: P_FAILURE"
        
    
    
    import onnxruntime as rt
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY]],
                              columns = ["MACHINE_ID", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
    inputArray.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)
    
    

    inputArray.drop(['MACHINE_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = rt.InferenceSession("/sasinside/userdata/gegrab/resources/hmeq/MACHINE_ONNX_XGBoost.onnx")
    
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    probabilities = sess.get_outputs()[1].name
    
    for x in test_data:
        pred_onnx = sess.run([probabilities], {input_name: np.expand_dims(x, axis=0).astype(np.float32)})[0]
    

    P_FAILURE = float(pred_onnx[0][0])
    
    
     
    return (P_FAILURE)"""



f = open('Data_orig/MACHINE_ONNX_XGB.py',"w+")
f.write(score_code)
f.close()

# Test XGBoost Score Code

In [22]:
def scoreXGB(MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY ):
    "Output: P_FAILURE"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY]],
                              columns = ["MACHINE_ID", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
    inputArray.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['MACHINE_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/MACHINE_XGBoost.pickle", 'rb'))

    P_FAILURE = float(sess.predict_proba(test_data)[0,0])
    
    
    #P_FAILURE = (test_data)[0,0]
    
     
     
    return (P_FAILURE)



In [23]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/machine2_ds2_test.csv') 
X_test2 
 
 
 
for i in range(5):
    print(scoreXGB(**X_test2.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/1639791302.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/1639791302.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

0.05046498775482178
0.09749722480773926
0.03160446882247925
0.1253322958946228
0.23529398441314697


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/1639791302.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/1639791302.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

# Write Out XGBoost Score Code

In [24]:
score_code = """
def scoreXGB(MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY ):
    "Output: P_FAILURE"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY]],
                              columns = ["MACHINE_ID", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
    inputArray.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['MACHINE_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/MACHINE_XGBoost.pickle", 'rb'))

    P_FAILURE = float(sess.predict_proba(test_data)[0,0])
    
     
     
    return (P_FAILURE)"""

f = open('Data_orig/MACHINE_XGB.py',"w+")
f.write(score_code)
f.close()

# Test Tensorflow Score Code

In [25]:
def scoreTF(MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY ):
    "Output: P_FAILURE"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    inputArray = pd.DataFrame([[MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY]],
                              columns = ["MACHINE_ID", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
    inputArray.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['MACHINE_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle") 
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
    
     
     
    sess = tf.keras.models.load_model("/sasinside/userdata/gegrab/resources/hmeq/MACHINE_Tensorflow.h5")
    
    P_FAILURE = float(sess.predict(test_data))
    P_FAIULRE = tf.keras.activations.sigmoid(P_FAILURE)
     
    
    return (P_FAILURE)



In [26]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/machine2_ds2_test.csv') 
X_test2 
 
 
 
for i in range(5):
    print(scoreTF(**X_test2.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3299353868.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


0.1901952028274536
0.1901952028274536


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3299353868.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3299353868.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

0.1901952028274536
0.1901952028274536


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3299353868.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_9143/3299353868.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

0.1901952028274536


# Write Out Tensorflow Score Code

In [27]:
score_code = """
def scoreTF(MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY ):
    "Output: P_FAILURE"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    inputArray = pd.DataFrame([[MACHINE_ID, DAILY_PRODUCTION, MONTHLY_PRODUCTION, PROD_TARGET, REASON, MONTHS_IN_OPERATION, NEGATIVE_RATING, DAYS_OUT_OF_SERVICE, MACHINE_AGE, SENSOR_INQUIRIES, NUM_SENSORS, SENSOR_RATIO, INDUSTRY]],
                              columns = ["MACHINE_ID", "DAILY_PRODUCTION", "MONTHLY_PRODUCTION", "PROD_TARGET", "REASON", "MONTHS_IN_OPERATION", "NEGATIVE_RATING", "DAYS_OUT_OF_SERVICE", "MACHINE_AGE", "SENSOR_INQUIRIES", "NUM_SENSORS", "SENSOR_RATIO", "INDUSTRY"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'TruckPr',regex = True, inplace=True)
    inputArray.INDUSTRY.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['MACHINE_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/machine_encoder.pickle") 
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/machine_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
    
    #print(test_data)
     
    sess = tf.keras.models.load_model("/sasinside/userdata/gegrab/resources/hmeq/MACHINE_Tensorflow.h5")
    
    P_FAILURE = float(sess.predict(test_data))
    P_FAIULRE = tf.keras.activations.sigmoid(P_FAILURE)
     
        
    return (P_FAILURE)"""


f = open('Data_orig/MACHINE_Tensorflow.py',"w+")
f.write(score_code)
f.close()


